In [19]:
from datasets import load_dataset

In [20]:
# https://oscar-project.org/post/oscar-2019/
dataset = load_dataset('oscar', 'unshuffled_deduplicated_ce')

Found cached dataset oscar (/home/moonstar/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_ce/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2)


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 2984
    })
})

In [22]:
dataset['train'][0]

{'id': 0,
 'text': 'Гренландин хӀорд — Къилбаседа Шен океанан дакъа, иза ду Гренландин, Исландин, Шпицберген а, Ян-Майен гӀайрена а юккъехь.'}

In [54]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 1000:
        # once we git the 10K mark, save to file
        with open(f'./Korean/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'./Korean/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))


  0%|          | 0/2984 [00:00<?, ?it/s]

In [49]:
from pathlib import Path

paths = [str(x) for x in Path('./').glob('text*.txt')]
paths

['text_2.txt', 'text_0.txt', 'text_1.txt']

In [50]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

In [51]:
tokenizer.train(files=paths[:20], vocab_size=1000, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [47]:
# tokenizer.train(files='text_3.txt', vocab_size=1000, min_frequency=2,
#                 special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

# import os

# os.mkdir('./english')

# tokenizer.save_model('english')

['english/vocab.json', 'english/merges.txt']

In [52]:
import os

os.mkdir('./filiberto')

tokenizer.save_model('filiberto')

['filiberto/vocab.json', 'filiberto/merges.txt']

In [34]:
from transformers import RobertaTokenizer

# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained('filiberto', max_len=512)

2022-12-29 15:09:30.235835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-29 15:09:30.420542: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-29 15:09:31.036119: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/home/moonstar/.mujoco/mujoco210/bin:/usr/lib/nvidia
2022-12-29 15:09:31.036207: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinf

In [35]:
# test our tokenizer on a simple sentence
tokens = tokenizer('ciao, come va?')

print(tokens)


{'input_ids': [0, 71, 77, 69, 83, 16, 225, 71, 83, 81, 73, 225, 90, 69, 35, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [36]:
tokens.input_ids

[0, 71, 77, 69, 83, 16, 225, 71, 83, 81, 73, 225, 90, 69, 35, 2]